In [1]:
import numpy as np

surface_data = np.load('surface_data.npy')
surface_labels = np.load('surface_data_labels.npy')
column_metadata = np.load('surface_columns_metadata.npy')

# Clasificador SVM

In [2]:
import pickle
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

# 5-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle = True)
clf = svm.SVC(kernel = 'linear')

acc = 0
recall = np.zeros(6)
precision = np.zeros(6)

for train_index, test_index in kf.split(surface_data, surface_labels):
    
    # Training phase
    x_train = surface_data[train_index, :]
    y_train = surface_labels[train_index]
    clf.fit(x_train, y_train)

    # Test phase
    x_test = surface_data[test_index, :]
    y_test = surface_labels[test_index]    
    y_pred = clf.predict(x_test)

    # Calculate confusion matrix and model performance
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion matrix\n', cm)
    
    diagonal_sum = 0
    for i in range(len(cm)):
        diagonal_sum += cm[i,i]
    
    acc += diagonal_sum/len(y_test)

    for i in range(len(cm)):
        current = cm[i, i]
        column_sum = 0
        for y in range(len(cm)):
            column_sum += cm[i, y]
        recall[i] = current / column_sum

    for i in range(len(cm)):
        current = cm[i, i]
        row_sum = 0
        for y in range(len(cm)):
            row_sum += cm[y, i]
        precision[i] = current / row_sum
    
    print('Iteration completed')

# Print results
acc = acc/5
print('Acc: ', acc)

precision = precision/5
print('Precision: ', precision)

recall = recall/5
print('Recall: ', recall)

# 5-fold cross-validation using cross_validate
cv_results = cross_validate(clf, surface_data, surface_labels, cv=5, scoring = ('accuracy', 'recall_micro'))
print('Acc: ', cv_results['test_accuracy'].sum()/5)
print('Recall: ', cv_results['test_recall_micro'].sum()/5)

with open('svm.pkl', 'wb') as f:
    pickle.dump(clf, f)


Confusion matrix
 [[110   2   0   0   0   0]
 [ 12   8   1   0   2   2]
 [ 18   1   2   1   2   1]
 [  1   2   1   3   0   2]
 [ 12   5   2   2   2   2]
 [  4   4   1   0   5  15]]
Iteration completed
Confusion matrix
 [[109   0   1   0   2   1]
 [ 10   6   3   0   3   3]
 [ 19   2   3   0   0   0]
 [  1   0   4   0   1   2]
 [ 11   2   1   1   3   7]
 [  6   0   0   0   5  18]]
Iteration completed
Confusion matrix
 [[108   1   1   0   0   2]
 [  9   7   5   2   0   2]
 [ 13   4   5   1   1   0]
 [  2   1   2   2   1   1]
 [ 15   2   0   1   2   6]
 [  4   5   1   0   2  16]]
Iteration completed
Confusion matrix
 [[99  1  4  4  2  2]
 [10 10  3  0  1  1]
 [13  2  7  0  2  0]
 [ 2  0  1  1  2  3]
 [ 9  2  0  0  6  8]
 [ 5  3  2  2  4 13]]
Iteration completed
Confusion matrix
 [[104   1   2   0   1   4]
 [ 11  10   1   0   1   2]
 [ 12   4   5   0   2   1]
 [  2   2   0   2   2   1]
 [  8   5   2   0   6   4]
 [  3   2   1   0   1  22]]
Iteration completed
Acc:  0.628015873015873
Precisi